In [32]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [33]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

$(p + \frac{a}{v^2})(v - b) = k_B T \quad \rightarrow \quad p = \frac{k_B T}{(v - b)} - \frac{a}{v^2}$

In [51]:
def get_absolute_isotherms(a, b, v_values, T_values):
    #given v (1d array) and T (Nd array) calculate N isotherm arrays 
    
    isotherms = []
    
    #N_A = 6.02214129e23 

    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1

    for T in T_values:
        isot = []
        for v in v_values:
            p = R*T/(v - b) - (a/v**2)
            isot.append(p)
            
        isotherms.append(isot)
        
    return isotherms

In [52]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b k_B}`
        
   #Args:
   #    a: Term related with the attraction between particles in
   #    J m^3/ mol^2.\n
   #    b: Term related with the volume that is occupied by one 
   #    mole of the molecules in 10^-3 m^3/mol.\n
   #    
   #Returns:
   #    p_c: Critical pressure in Pa.\n
   #    v_c: Critical volume in m^3/mol.\n
   #    T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/R
    
    return p_c, v_c, T_c

In [70]:
def update_isotherms(change):
    
    a = a_slider.value
    b = b_slider.value
    
    isotherms = get_absolute_isotherms(a, b, v_values, T_values)
    
    marks = []
    middle_index = int(len(fig_114_001.marks)/2)
    
    for i in range(len(isotherms)):
    
        marks.append(bqm.Lines(
                    x = v_values, 
                    y = isotherms[i], 
                    scales = {'x': scale_x, 'y': scale_y}, 
                    opacities = [1.0],
                    visible = True, #True, #t == '1.00',
                    colors = ['red'],
                    labels = [str(T_values[i])],
                    ))
    
    for i in range(middle_index, len(fig_114_001.marks)):
        
        marks.append(fig_114_001.marks[i])
        
    fig_114_001.marks = marks

In [79]:
def restart(a):
    
    a_slider.value = a_initial
    b_slider.value = b_initial

In [81]:
a_initial = 5.536 #L^2 bar/mol^2
b_initial = 0.03049 #L/mol

a, b = a_initial, b_initial

p_c, v_c, T_c = calculate_critic(a, b)

T_values = [0.6*T_c, 0.8*T_c, T_c, 1.2*T_c]
v_values = np.linspace(0.8*b, 10*v_c, 500)

isotherms = get_absolute_isotherms(a, b, v_values, T_values)

#######################################
#######CREATE THE FIGURE###############
#######################################


fig_114_001 = bq.Figure(title='',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=80, right=30),
                toolbar = True
    )

scale_x = bqs.LinearScale(min = min(v_values), max = max(v_values))
scale_y = bqs.LinearScale(min = 0.0, max = 2.0*p_c)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_114_001.axes = [axis_x, axis_y]

marks = []

for i in range(len(isotherms)):
    
    marks.append(bqm.Lines(
                x = v_values, 
                y = isotherms[i], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = ['red'],
                labels = [str(T_values[i])],
                ))
    
for i in range(len(isotherms)):
    
    marks.append(bqm.Lines(
                x = v_values, 
                y = isotherms[i], 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = [0.2],
                visible = True, #True, #t == '1.00',
                colors = ['red'],
                labels = [str(T_values[i])],
                ))
    
fig_114_001.marks = marks



a_slider = widgets.FloatSlider(
    min=0.0,
    max=2.0*a,
    step=0.1,
    value=a,
    description='a',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

a_slider.observe(update_isotherms, 'value')

b_slider = widgets.FloatSlider(
    min=0.0,
    max=4.0*b,
    step=0.001,
    value=b,
    description='b',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

b_slider.observe(update_isotherms, 'value')

return_button = widgets.Button(
    description='Return',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Return to the original state',
)

return_button.on_click(restart)

top_block = widgets.VBox([], layout=widgets.Layout(width='50%', align_self='center'))

top_block.children = [fig_114_001, a_slider, b_slider, return_button]
top_block

In [67]:
fig_114_001

Figure(axes=[Axis(grid_color='#8e8e8e', grid_lines='none', label='v', label_offset='50px', num_ticks=5, scale=…

In [59]:
fig_114_001.marks[2].y

array([-1.81268875e+04, -2.05505378e+04, -2.83483259e+04, -7.84257983e+04,
        4.62224973e+04,  1.40689968e+04,  7.18293349e+03,  4.34746688e+03,
        2.87743076e+03,  2.01807355e+03,  1.47727496e+03,  1.11960685e+03,
        8.74493212e+02,  7.02008482e+02,  5.78152340e+02,  4.87808144e+02,
        4.21096355e+02,  3.71364146e+02,  3.34019713e+02,  3.05827749e+02,
        2.84468585e+02,  2.68254097e+02,  2.55939928e+02,  2.46598640e+02,
        2.39532354e+02,  2.34211568e+02,  2.30231651e+02,  2.27281504e+02,
        2.25120697e+02,  2.23562621e+02,  2.22461939e+02,  2.21705166e+02,
        2.21203537e+02,  2.20887560e+02,  2.20702840e+02,  2.20606854e+02,
        2.20566436e+02,  2.20555826e+02,  2.20555127e+02,  2.20549100e+02,
        2.20526203e+02,  2.20477834e+02,  2.20397726e+02,  2.20281465e+02,
        2.20126105e+02,  2.19929856e+02,  2.19691840e+02,  2.19411891e+02,
        2.19090393e+02,  2.18728153e+02,  2.18326296e+02,  2.17886186e+02,
        2.17409353e+02,  